In [5]:
!pip install -q tensorflow-addons
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os 
import csv
import pickle

from sklearn.metrics import confusion_matrix, classification_report


import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Activation

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19


AUTOTUNE = tf.data.experimental.AUTOTUNE
from typeguard import typechecked
from typing import Optional

import warnings
warnings.filterwarnings("ignore")

print(f"All OK!\ntf version {tf.__version__}")

All OK!
tf version 2.8.0


In [ ]:
tfa.__version__

'0.14.0'

In [6]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
def ValPlot(acc,val_acc,loss,val_loss,auc,val_auc,precision,val_precision,f1,val_f1,model_no):    
    plt.figure(figsize=(16,8)) 
    plt.plot(range(1, len(acc) + 1), acc)
    plt.plot(range(1, len(val_acc) + 1), val_acc)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(['training', 'validation'])
    plt.title('History of Accuracy')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-accuracy-graph')

    plt.figure(figsize=(16,8)) 
    plt.plot(range(1, len(loss) + 1), loss)
    plt.plot(range(1, len(val_loss) + 1), val_loss)
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend(['training', 'validation'])
    plt.title('History of Loss')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-loss-graph')

    plt.figure(figsize=(16,8)) 
    plt.plot(range(1, len(auc) + 1), auc)
    plt.plot(range(1, len(val_auc) + 1), val_auc)
    plt.xlabel('Epochs')
    plt.ylabel('AUC')
    plt.legend(['training', 'validation'])
    plt.title('History of AUC')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-auc-graph')


    plt.figure(figsize=(16,8)) 
    plt.plot(range(1, len(precision) + 1), precision)
    plt.plot(range(1, len(val_precision) + 1), val_precision)
    plt.xlabel('Epochs')
    plt.ylabel('Precision')
    plt.legend(['training', 'validation'])
    plt.title('History of Precision')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-pre-graph')

    plt.figure(figsize=(16,8)) 
    plt.plot(range(1, len(f1) + 1), f1)
    plt.plot(range(1, len(val_f1) + 1), val_f1)
    plt.xlabel('Epochs')
    plt.ylabel('F1-Score')
    plt.legend(['training', 'validation'])
    plt.title('History of F1-Score')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-f1-graph')

    """
    fig, (ax1, ax2,ax3,ax4,ax5) = plt.subplots(1,5, figsize= (30,5))
    fig.suptitle(" MODEL'S METRICS VISUALIZATION ")

    ax1.plot(range(1, len(acc) + 1), acc)
    ax1.plot(range(1, len(val_acc) + 1), val_acc)
    ax1.set_title('History of Accuracy')
    ax1.set_xlabel('Epochs')
    ax1.set_ylabel('Accuracy')
    ax1.legend(['training', 'validation'])


    ax2.plot(range(1, len(loss) + 1), loss)
    ax2.plot(range(1, len(val_loss) + 1), val_loss)
    ax2.set_title('History of Loss')
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss')
    ax2.legend(['training', 'validation'])
    
    ax3.plot(range(1, len(auc) + 1), auc)
    ax3.plot(range(1, len(val_auc) + 1), val_auc)
    ax3.set_title('History of AUC')
    ax3.set_xlabel('Epochs')
    ax3.set_ylabel('AUC')
    ax3.legend(['training', 'validation'])
    
    ax4.plot(range(1, len(precision) + 1), precision)
    ax4.plot(range(1, len(val_precision) + 1), val_precision)
    ax4.set_title('History of Precision')
    ax4.set_xlabel('Epochs')
    ax4.set_ylabel('Precision')
    ax4.legend(['training', 'validation'])
    
    ax5.plot(range(1, len(f1) + 1), f1)
    ax5.plot(range(1, len(val_f1) + 1), val_f1)
    ax5.set_title('History of F1-score')
    ax5.set_xlabel('Epochs')
    ax5.set_ylabel('F1 score')
    ax5.legend(['training', 'validation'])
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/graph/model_{model_no}-train-val-graph')
    plt.show()
    """

In [8]:
def model_report(model, test_dataset, model_no):
    print("Model Evaluating !!")
    scores = model.evaluate_generator(test_dataset)
    metrics = ["Loss", "Accuracy", "Precision", "Recall", "AUC", "F1_score"]
    for m, s in zip(metrics, scores):
        print(f"{m}: {s}")

    print("Report CSV Generating !!")
    with open(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/report/model_{model_no}_report.csv', 'w') as f:
        writer = csv.writer(f)
        writer.writerow(metrics)
        writer.writerow(scores)
    
    print("Confusion Matrix Generating !!")
    pred = model.predict(test_dataset)
    y_pred = np.argmax(pred, axis=1)
    y_true = test_dataset.classes
    CLASSES = list(test_dataset.class_indices.keys())


    print(classification_report(y_true, y_pred, target_names=CLASSES))

    conf_arr = confusion_matrix(y_true, y_pred)

    plt.figure(figsize=(8, 6), dpi=80, facecolor='w', edgecolor='k')
    ax = sns.heatmap(conf_arr, cmap='Greens', annot=True, fmt='d', xticklabels=CLASSES, yticklabels=CLASSES)
    plt.title('Alzheimer\'s Disease Diagnosis')
    plt.xlabel('Prediction')
    plt.ylabel('Truth')
    plt.savefig(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/report/model_{model_no}-confusion_matrix')
    plt.show(ax)

In [9]:
def make_dataset(train_dir, test_dir):
    train_datagen = ImageDataGenerator(
            rescale = 1./255,
            validation_split = 0.2,
            rotation_range=5,
            width_shift_range=0.2,
            height_shift_range=0.2,
            shear_range=0.2,
            #zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,
            fill_mode='nearest')

    valid_datagen = ImageDataGenerator(
        rescale = 1./255,
        validation_split = 0.2)

    test_datagen  = ImageDataGenerator(rescale = 1./255)

    train_dataset  = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (224,224),
                                                    class_mode = 'categorical',
                                                    subset = 'training',
                                                    batch_size = 32)
    valid_dataset = valid_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (224,224),
                                                    class_mode = 'categorical',
                                                    subset = 'validation',
                                                    batch_size = 32)
    test_dataset = test_datagen.flow_from_directory(directory = test_dir,
                                                    target_size = (224,224),
                                                    class_mode = 'categorical',
                                                    batch_size = 32)
    return train_dataset, valid_dataset, test_dataset

In [10]:
def make_model(base_model, trainable):
    for layer in base_model.layers:
        layer.trainable = trainable

    custom_model = Sequential([
        base_model, #<----- change here 
        Dropout(0.5),
        Flatten(),
        BatchNormalization(),
        Dense(64,kernel_initializer='he_uniform'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(64,kernel_initializer='he_uniform'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(64,kernel_initializer='he_uniform'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(32,kernel_initializer='he_uniform'),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(32,kernel_initializer='he_uniform'),
        BatchNormalization(),
        Activation('relu'),
        Dense(4,activation='softmax')    
    ], name = "custom_cnn_model")
    return custom_model

In [11]:
def train_model(model, optimizer, train_dataset, valid_dataset, model_no):
    initial_learning_rate = 0.01
    decay_rate = 0.1
    num_epoch = 10

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=num_epoch,
        decay_rate=decay_rate,
        staircase=True)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall'),  
        tf.keras.metrics.AUC(name='auc'),
        tfa.metrics.F1Score(num_classes=4, average='micro')
    ]
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=METRICS)
    history=model.fit(
        train_dataset,
        validation_data=valid_dataset,
        epochs=num_epoch,
        verbose=1,
        callbacks=lr_scheduler
    )
    model.save(f"/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model_{model_no}.h5")
    print(f"Saved model no. {model_no} to disk")
    return model, history

In [12]:
train_dir = '/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/train'
test_dir = '/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/val'
train_ables = [True, False]
model_no = 0

optimizers = ['rmsprop', tf.keras.optimizers.Adam(1e-4)]

base_models = [
               InceptionV3(input_shape=(224,224,3), include_top=False, weights="imagenet"),
               ResNet50(input_shape=(224,224,3), include_top=False, weights="imagenet"),
               VGG16(input_shape=(224,224,3), include_top=False, weights="imagenet"),
               VGG19(input_shape=(224,224,3),include_top=False, weights="imagenet")
]

80150528/80134624 [==============================] - 1s 0us/step


In [ ]:
for base_model in base_models:
    for optimizer in optimizers:
        for train_able in train_ables:
            if not os.path.exists(f"/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model_{model_no}.h5"):
                print(f"Model no {model_no} is Trainning !!")
                train_dataset, valid_dataset, test_dataset = make_dataset(train_dir, test_dir)
                model = make_model(base_model, train_able)
                #model.summary()
                model, history = train_model(model, optimizer, train_dataset, valid_dataset, model_no)
            else:
                #model = load_model(f"/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model_{model_no}.h5")
                print(f"Model no {model_no} already Trainned !!")
            if not os.path.exists(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model_{model_no}-train-val-graph.png'):
                ValPlot(history.history['accuracy'],history.history['val_accuracy'],
                    history.history['loss'],history.history['val_loss'],
                    history.history['auc'],history.history['val_auc'],
                    history.history['precision'],history.history['val_precision'],
                    history.history['f1_score'],history.history['val_f1_score'],
                    model_no = model_no
                )
            if not os.path.exists(f'/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model_{model_no}-confusion_matrix.png'):
                model_report(model, test_dataset, model_no)
            model_no += 1

Model no 0 already Trainned !!
Model no 1 already Trainned !!
Model no 2 already Trainned !!
Model no 3 already Trainned !!
Model no 4 already Trainned !!
Model no 5 already Trainned !!
Model no 6 already Trainned !!
Model no 7 already Trainned !!
Model no 8 already Trainned !!
Model no 9 already Trainned !!
Model no 10 already Trainned !!
Model no 11 is Trainning !!
Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
Epoch 1/10
  9/129 [=>............................] - ETA: 22:21 - loss: 1.6512 - accuracy: 0.7179 - precision: 0.2771 - recall: 0.0799 - auc: 0.4916 - f1_score: 0.2743

KeyboardInterrupt: ignored

In [ ]:
base_model = ResNet50(input_shape=(224,224,3), include_top=False, weights="imagenet")
train_able = True
optimizer = tf.keras.optimizers.Adam(1e-4)

train_dataset, valid_dataset, test_dataset = make_dataset(train_dir, test_dir)
model = make_model(base_model, train_able)
model, history = train_model(model, optimizer, train_dataset, valid_dataset, model_no)

file_loc = "/content/gdrive/MyDrive/Colab Notebooks/Alzheimer/models/model.pkl"
with open(file_loc, 'wb') as f:
    pickle.dump(history, f)

ValPlot(history.history['accuracy'],history.history['val_accuracy'],
                    history.history['loss'],history.history['val_loss'],
                    history.history['auc'],history.history['val_auc'],
                    history.history['precision'],history.history['val_precision'],
                    history.history['f1_score'],history.history['val_f1_score'],
                    model_no = 100
                )

Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.
Epoch 1/10
 98/129 [=====================>........] - ETA: 7:42 - loss: 1.1223 - accuracy: 0.7424 - precision: 0.4672 - recall: 0.2180 - auc: 0.7562 - f1_score: 0.4601